Importing all the necessary libraries for the ELT

In [1]:
import pandas as pd

In [2]:
import pyorc

In [3]:
from datetime import datetime

Read file

In [4]:
example = open("trades.orc", 'rb')

In [5]:
reader = pyorc.Reader(example)

In [6]:
schema = reader.schema

In [7]:
columns = schema.fields

In [8]:
df=pd.DataFrame(reader, columns=columns, dtype=str)

In [9]:
df

,trade_id,price,quantity,time,isSelling
0,1117732504.0,1793.61,0.0201,1680220800001.0,False
1,1117732505.0,1793.6,0.0197,1680220800001.0,True
2,1117732506.0,1793.61,0.038,1680220800005.0,False
3,1117732507.0,1793.6,0.0222,1680220800005.0,True
4,1117732508.0,1793.61,0.0209,1680220800005.0,False
...,...,...,...,...,...
902950,1118635454.0,1821.51,0.0061,1680307199996.0,True
902951,1118635455.0,1821.51,0.0153,1680307199997.0,True
902952,1118635456.0,1821.51,0.0341,1680307199997.0,True
902953,1118635457.0,1821.51,0.033,1680307199999.0,True


In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902955 entries, 0 to 902954
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   trade_id   902954 non-null  object
 1   price      902954 non-null  object
 2   quantity   902954 non-null  object
 3   time       902954 non-null  object
 4   isSelling  902954 non-null  object
dtypes: object(5)
memory usage: 34.4+ MB
None


Prepare the DataFreame


In [11]:
df = df.rename(columns={'trade_id': 'trade_id_src'})

In [12]:
df = df.rename(columns={'price': 'price_src'})

In [13]:
df = df.rename(columns={'quantity': 'quantity_src'})

In [14]:
df = df.rename(columns={'time': 'time_src'})

In [15]:
df = df.rename(columns={'isSelling': 'is_selling_src'})

Clean and customize

In [16]:
df['trade_id_str'] = df['trade_id_src'].str.slice(stop=-2)

In [17]:
df['price_fl'] = df['price_src'].astype(float)

In [18]:
df['quantity_fl'] = df['quantity_src'].astype(float)

In [19]:
df['time_dt'] = pd.to_datetime(df['time_src'].astype(float)/1000, unit='s')


In [20]:
df['is_selling_bl'] = df['is_selling_src'].str.lower() == 'true'

In [21]:
df = df.sort_values('time_dt')

In [22]:
df = df.dropna()

In [23]:
df = df.drop_duplicates(subset=['price_fl', 'quantity_fl', 'time_dt', 'is_selling_bl' ], keep='first')

In [24]:
df

,trade_id_src,price_src,quantity_src,time_src,is_selling_src,trade_id_str,price_fl,quantity_fl,time_dt,is_selling_bl
0,1117732504.0,1793.61,0.0201,1680220800001.0,False,1117732504,1793.61,0.0201,2023-03-31 00:00:00.000999936,False
1,1117732505.0,1793.6,0.0197,1680220800001.0,True,1117732505,1793.60,0.0197,2023-03-31 00:00:00.000999936,True
2,1117732506.0,1793.61,0.038,1680220800005.0,False,1117732506,1793.61,0.0380,2023-03-31 00:00:00.005000192,False
3,1117732507.0,1793.6,0.0222,1680220800005.0,True,1117732507,1793.60,0.0222,2023-03-31 00:00:00.005000192,True
4,1117732508.0,1793.61,0.0209,1680220800005.0,False,1117732508,1793.61,0.0209,2023-03-31 00:00:00.005000192,False
...,...,...,...,...,...,...,...,...,...,...
902950,1118635454.0,1821.51,0.0061,1680307199996.0,True,1118635454,1821.51,0.0061,2023-03-31 23:59:59.996000000,True
902951,1118635455.0,1821.51,0.0153,1680307199997.0,True,1118635455,1821.51,0.0153,2023-03-31 23:59:59.996999936,True
902952,1118635456.0,1821.51,0.0341,1680307199997.0,True,1118635456,1821.51,0.0341,2023-03-31 23:59:59.996999936,True
902953,1118635457.0,1821.51,0.033,1680307199999.0,True,1118635457,1821.51,0.0330,2023-03-31 23:59:59.999000064,True


In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 888619 entries, 0 to 902954
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   trade_id_src    888619 non-null  object        
 1   price_src       888619 non-null  object        
 2   quantity_src    888619 non-null  object        
 3   time_src        888619 non-null  object        
 4   is_selling_src  888619 non-null  object        
 5   trade_id_str    888619 non-null  object        
 6   price_fl        888619 non-null  float64       
 7   quantity_fl     888619 non-null  float64       
 8   time_dt         888619 non-null  datetime64[ns]
 9   is_selling_bl   888619 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(6)
memory usage: 68.6+ MB
None


Index and Audit fields

In [26]:
df['last_modified_dt'] = datetime.now()

In [27]:
df = df.reset_index(drop=True)

In [28]:
df

,trade_id_src,price_src,quantity_src,time_src,is_selling_src,trade_id_str,price_fl,quantity_fl,time_dt,is_selling_bl,last_modified_dt
0,1117732504.0,1793.61,0.0201,1680220800001.0,False,1117732504,1793.61,0.0201,2023-03-31 00:00:00.000999936,False,2023-04-15 18:03:47.083206
1,1117732505.0,1793.6,0.0197,1680220800001.0,True,1117732505,1793.60,0.0197,2023-03-31 00:00:00.000999936,True,2023-04-15 18:03:47.083206
2,1117732506.0,1793.61,0.038,1680220800005.0,False,1117732506,1793.61,0.0380,2023-03-31 00:00:00.005000192,False,2023-04-15 18:03:47.083206
3,1117732507.0,1793.6,0.0222,1680220800005.0,True,1117732507,1793.60,0.0222,2023-03-31 00:00:00.005000192,True,2023-04-15 18:03:47.083206
4,1117732508.0,1793.61,0.0209,1680220800005.0,False,1117732508,1793.61,0.0209,2023-03-31 00:00:00.005000192,False,2023-04-15 18:03:47.083206
...,...,...,...,...,...,...,...,...,...,...,...
888614,1118635454.0,1821.51,0.0061,1680307199996.0,True,1118635454,1821.51,0.0061,2023-03-31 23:59:59.996000000,True,2023-04-15 18:03:47.083206
888615,1118635455.0,1821.51,0.0153,1680307199997.0,True,1118635455,1821.51,0.0153,2023-03-31 23:59:59.996999936,True,2023-04-15 18:03:47.083206
888616,1118635456.0,1821.51,0.0341,1680307199997.0,True,1118635456,1821.51,0.0341,2023-03-31 23:59:59.996999936,True,2023-04-15 18:03:47.083206
888617,1118635457.0,1821.51,0.033,1680307199999.0,True,1118635457,1821.51,0.0330,2023-03-31 23:59:59.999000064,True,2023-04-15 18:03:47.083206


In [29]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888619 entries, 0 to 888618
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   trade_id_src      888619 non-null  object        
 1   price_src         888619 non-null  object        
 2   quantity_src      888619 non-null  object        
 3   time_src          888619 non-null  object        
 4   is_selling_src    888619 non-null  object        
 5   trade_id_str      888619 non-null  object        
 6   price_fl          888619 non-null  float64       
 7   quantity_fl       888619 non-null  float64       
 8   time_dt           888619 non-null  datetime64[ns]
 9   is_selling_bl     888619 non-null  bool          
 10  last_modified_dt  888619 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(2), object(6)
memory usage: 68.6+ MB
None


Find Metrics

In [30]:
df['total_fl'] = df.apply(lambda row: -1 if row['is_selling_bl'] == False else 1, axis=1) * df['quantity_fl'].values * df['price_fl']

In [31]:
bot_outcome = df['total_fl'].sum()

In [32]:
df['rowcount_in'] = df['is_selling_bl'].where(df['is_selling_bl'], 0).astype(int).groupby((df['is_selling_bl'] != df['is_selling_bl'].shift()).cumsum()).cumsum()

In [33]:
longest_chain = df['rowcount_in'].max()

In [34]:
max_index = df['rowcount_in'].idxmax()

In [35]:
min_index = (pd.to_numeric(max_index)-pd.to_numeric(longest_chain))+1

In [36]:
df.loc[min_index, 'rowcount_in']

1

In [37]:
chain_price_diff = df.loc[max_index, 'price_fl']-df.loc[min_index, 'price_fl']

Results

In [38]:
df

,trade_id_src,price_src,quantity_src,time_src,is_selling_src,trade_id_str,price_fl,quantity_fl,time_dt,is_selling_bl,last_modified_dt,total_fl,rowcount_in
0,1117732504.0,1793.61,0.0201,1680220800001.0,False,1117732504,1793.61,0.0201,2023-03-31 00:00:00.000999936,False,2023-04-15 18:03:47.083206,-36.051561,0
1,1117732505.0,1793.6,0.0197,1680220800001.0,True,1117732505,1793.60,0.0197,2023-03-31 00:00:00.000999936,True,2023-04-15 18:03:47.083206,35.333920,1
2,1117732506.0,1793.61,0.038,1680220800005.0,False,1117732506,1793.61,0.0380,2023-03-31 00:00:00.005000192,False,2023-04-15 18:03:47.083206,-68.157180,0
3,1117732507.0,1793.6,0.0222,1680220800005.0,True,1117732507,1793.60,0.0222,2023-03-31 00:00:00.005000192,True,2023-04-15 18:03:47.083206,39.817920,1
4,1117732508.0,1793.61,0.0209,1680220800005.0,False,1117732508,1793.61,0.0209,2023-03-31 00:00:00.005000192,False,2023-04-15 18:03:47.083206,-37.486449,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
888614,1118635454.0,1821.51,0.0061,1680307199996.0,True,1118635454,1821.51,0.0061,2023-03-31 23:59:59.996000000,True,2023-04-15 18:03:47.083206,11.111211,1
888615,1118635455.0,1821.51,0.0153,1680307199997.0,True,1118635455,1821.51,0.0153,2023-03-31 23:59:59.996999936,True,2023-04-15 18:03:47.083206,27.869103,2
888616,1118635456.0,1821.51,0.0341,1680307199997.0,True,1118635456,1821.51,0.0341,2023-03-31 23:59:59.996999936,True,2023-04-15 18:03:47.083206,62.113491,3
888617,1118635457.0,1821.51,0.033,1680307199999.0,True,1118635457,1821.51,0.0330,2023-03-31 23:59:59.999000064,True,2023-04-15 18:03:47.083206,60.109830,4


In [39]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888619 entries, 0 to 888618
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   trade_id_src      888619 non-null  object        
 1   price_src         888619 non-null  object        
 2   quantity_src      888619 non-null  object        
 3   time_src          888619 non-null  object        
 4   is_selling_src    888619 non-null  object        
 5   trade_id_str      888619 non-null  object        
 6   price_fl          888619 non-null  float64       
 7   quantity_fl       888619 non-null  float64       
 8   time_dt           888619 non-null  datetime64[ns]
 9   is_selling_bl     888619 non-null  bool          
 10  last_modified_dt  888619 non-null  datetime64[ns]
 11  total_fl          888619 non-null  float64       
 12  rowcount_in       888619 non-null  int32         
dtypes: bool(1), datetime64[ns](2), float64(3), int32(1), object

In [40]:
print('Bot outcome for the day:', bot_outcome)
print('Longest chain of consecutive sells:', longest_chain)
print('Price difference between first and last operation of longest chain:', chain_price_diff)

Bot outcome for the day: -41614981.018217005
Longest chain of consecutive sells: 485
Price difference between first and last operation of longest chain: 2.9500000000000455
